### This notebook is the continuation of the featurization_notebook where we got the features and saved them to a file.

In [1]:
import sys
sys.path.append('../modules')
import likelihood_predictor
from likelihood_predictor import PlastPredictor
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.stats import zscore
import pickle
from sklearn.feature_selection import SelectFromModel

In [2]:
pl_full = pd.read_pickle('../database/plasticizer_data_v10_polarity.pkl')
pl_pol = pd.concat([pl_full[pl_full.columns[1:195]], pl_full['Polarity']], axis=1)
all_cols = pl_pol.columns.to_numpy()
pl_data = pl_pol[all_cols].to_numpy()
lin_data = pd.read_pickle('../database/linolein_test.pkl')
lin_data['Polarity'] = 0.048856
lin_data = lin_data[all_cols].to_numpy()
org_full = pd.read_pickle('../database/org_polarity_v2.pkl')

In [3]:
psim1 = open("pubs_similarity.txt", 'r')
psim11 = [line.rstrip('\n') for line in psim1]
psim2 = open("pubs_othersim.txt", 'r')
psim22 = [line.rstrip('\n') for line in psim2]

In [4]:
org_full
org_full['Dsim'] = psim11
org_full['Nasim'] = psim22
org_full = org_full.sort_values(by ='Dsim')

In [5]:
org_full = org_full[:5000]
org_data = org_full[all_cols]

In [7]:
pl_full = pd.read_excel('feature_final_selected.xls')

In [8]:
pl_full

,Unnamed: 0,0
0,25,-1.060000
1,5,0.000000
2,65,0.000000
3,110,0.000000
4,132,0.000000
5,177,0.000000
6,149,2.000000
7,101,4.000000
8,0,11.863075
9,44,13.213764


This is our df. Ignore the last column. We are focusing on the **zero** column, which are the positions of our relevant features.

In [9]:
pl_full.columns = ['feat_num', 'pl']

In [10]:
pl_full = pl_full.sort_values(by='feat_num', ascending=True)

In [11]:
pl_list = pl_full['pl'].to_list()

In [12]:
pl_full

,feat_num,pl
8,0,11.863075
1,5,0.000000
0,25,-1.060000
14,41,91.903165
9,44,13.213764
10,51,13.213764
2,65,0.000000
12,72,37.891219
13,73,64.208216
11,86,16.536151


In [13]:
positions = pl_full['feat_num'].to_list()

In [14]:
positions

[0, 5, 25, 41, 44, 51, 65, 72, 73, 86, 101, 110, 132, 149, 177]

In [15]:
feat = pl_pol.columns.to_list()

In [16]:
sel_ft = []
for i in positions:
    sel_ft.append(feat[i])

In [17]:
sel_ft

['MaxEStateIndex',
 'NumRadicalElectrons',
 'HallKierAlpha',
 'PEOE_VSA6',
 'PEOE_VSA9',
 'SMR_VSA6',
 'SlogP_VSA8',
 'EState_VSA3',
 'EState_VSA4',
 'VSA_EState7',
 'NumHeteroatoms',
 'fr_Al_OH',
 'fr_aldehyde',
 'fr_ester',
 'fr_phos_ester']

In [18]:
pl_pol=pl_pol.T

In [19]:
pl_new = pl_pol.loc[sel_ft].T

In [20]:
pl_new

,MaxEStateIndex,NumRadicalElectrons,HallKierAlpha,PEOE_VSA6,PEOE_VSA9,SMR_VSA6,SlogP_VSA8,EState_VSA3,EState_VSA4,VSA_EState7,NumHeteroatoms,fr_Al_OH,fr_aldehyde,fr_ester,fr_phos_ester
0,11.863075,0.0,-1.06,91.903165,13.213764,13.213764,0.000000,37.891219,64.208216,16.536151,4.0,0.0,0.0,2.0,0.0
1,12.421299,0.0,-1.84,91.194256,24.340667,13.213764,0.000000,24.340667,50.360742,11.074329,4.0,0.0,0.0,2.0,0.0
2,11.276404,0.0,-0.78,48.530937,9.790967,0.000000,21.544897,17.193270,21.899351,0.501610,10.0,0.0,0.0,0.0,0.0
3,11.944745,0.0,-1.84,38.821852,24.340667,13.213764,0.000000,13.213764,25.683286,3.533412,4.0,0.0,0.0,2.0,0.0
4,11.268799,0.0,-1.06,27.694949,13.213764,13.213764,0.000000,50.732863,0.000000,2.106920,4.0,0.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,12.960039,0.0,-2.37,117.083536,36.511000,19.820646,0.000000,13.213764,57.787395,19.558226,6.0,0.0,0.0,3.0,0.0
206,5.117870,0.0,-2.28,54.616610,6.263163,0.000000,17.045264,0.000000,17.390066,7.195278,1.0,0.0,0.0,0.0,0.0
207,11.861333,0.0,-5.38,412.944246,0.000000,52.855056,0.000000,25.683286,64.208216,81.069210,22.0,2.0,0.0,10.0,0.0
208,12.889740,0.0,-1.46,19.056471,6.420822,29.749429,0.000000,47.770359,62.362365,9.929419,7.0,0.0,0.0,1.0,0.0


In [21]:
org_data = org_data.T

In [22]:
org_new = org_data.loc[sel_ft].T

In [23]:
org_new.to_excel('new_selected_org.xls')

In [24]:
pl_new.to_excel('new_selected_pl.xls')

In [25]:
org_num = org_new.to_numpy()

In [26]:
pl_num = pl_new.to_numpy()

In [27]:
reg_param = 10
pp = PlastPredictor(reg_param)
pp_model = pp.fit_model(pl_num, org_num)


In [28]:
org_acc = pp.predict(org_num, type='binary', class_id='neg')
pl_acc = pp.predict(pl_num, type='binary', class_id='pos')
cc=pp.clf.coef_

In [29]:
np.count_nonzero(cc), org_acc, pl_acc

(15, 0.9554, 0.8952380952380953)

In [30]:
sel_ft

['MaxEStateIndex',
 'NumRadicalElectrons',
 'HallKierAlpha',
 'PEOE_VSA6',
 'PEOE_VSA9',
 'SMR_VSA6',
 'SlogP_VSA8',
 'EState_VSA3',
 'EState_VSA4',
 'VSA_EState7',
 'NumHeteroatoms',
 'fr_Al_OH',
 'fr_aldehyde',
 'fr_ester',
 'fr_phos_ester']